In [1]:
import cc3d
import ants
import numpy as np

def skull_id(labels_out, idx):
    labels_out = labels_out.reshape((1,-1))
    labels_out = labels_out[0,:]
    label = np.unique(labels_out)
    label = np.arange(np.max(label) + 2)
    hist, bin_edges = np.histogram(labels_out, bins=label)
    hist = np.ndarray.tolist(hist)
    hist_ = hist
    hist_ = np.array(hist_)
    hist.sort(reverse=True)
    #print('hist',hist)
    idx = (hist_== hist[idx])
    idx = idx + 1 - 1
    idx_ = np.sum(idx * label[0:len(idx)])
#     print('implant_idx = ',idx_)
    return idx_

def cca_processing(data, idx):
    # original data (512,512,z)
    # idx refers to which components you want to choose
    labels_out = cc3d.connected_components(data.astype('int32'))
    skull_label = skull_id(labels_out, idx)
    skull = (labels_out == skull_label)
    skull=(skull + 1 - 1).astype('float32')
    return skull

def clean_operation(pred, defect):
    # clean opeation to remove those voxels which original defects and implant both have.
    pred[pred == defect] = 0
    # ants could only process float datatype
    pred = pred.astype('float32')
    return pred

In [45]:
idx = 11
pred_implant_name = './100MixedCases/pred_implant/{}_averaged_implant.nrrd'.format(str(idx).zfill(2))
original_defect_name = './OriginalSkull/sub{}.nrrd'.format(str(idx).zfill(2))
implant = ants.image_read(pred_implant_name)

defect_skull = ants.image_read(original_defect_name)
origin, spacing, direction = defect_skull.origin, defect_skull.spacing, defect_skull.direction


# Step 1
implant = ants.morphology(implant, 'open', 4)
implant = cca_processing(implant.numpy(), 1)
implant = ants.from_numpy(implant, origin=origin, spacing=spacing, direction=direction)

# Step 2
implant = clean_operation(implant.numpy(), defect_skull.numpy())
implant = ants.from_numpy(implant, origin=origin, spacing=spacing, direction=direction)

output_name = './OriginalSkull/sub{}_output_implant.nrrd'.format(str(idx).zfill(2))
ants.image_write(implant, output_name)
print("save:" + str(idx))        

save:11


In [20]:
# Method to remove noise that is diffifult to be removed only using opening operation
# "a" and "b" refer to results of differernt opening parameters. 
# Usually, radius parameters of "b" is smaller than "a". It depends to how you want to process the data

idx = 3
b = ants.image_read('./OriginalSkull/sub{}_output_implant_R3.nrrd'.format(str(idx).zfill(2)))
a = ants.image_read('./OriginalSkull/sub{}_output_implant_R4.nrrd'.format(str(idx).zfill(2)))

noise = b > a
origin, spacing, direction = b.origin, b.spacing, b.direction

# Here index of components must be set manually, because the noise may not always be the biggest part.
noise = cca_processing(noise.numpy(), 1)
noise = ants.from_numpy(noise, origin=origin, spacing=spacing, direction=direction)

b[b == noise] = 0
output_name = './OriginalSkull/sub{}_output_implant2.nrrd'.format(str(idx).zfill(2))
ants.image_write(b, output_name)
print("save:" + str(idx)) 

save:3
